In [1]:
# scikit-surprise
# https://learn.microsoft.com/en-US/cpp/windows/latest-supported-vc-redist?view=msvc-170#visual-studio-2015-2017-2019-and-2022
# https://stackoverflow.com/questions/61365790/error-could-not-build-wheels-for-scipy-which-use-pep-517-and-cannot-be-installe
# pip install --upgrade pip setuptools wheel / pip3 install scikit-surprise

In [2]:
import os
import sqlalchemy
import pandas as pd
from dotenv import load_dotenv 
from surprise import Reader, Dataset, SVD
from sqlalchemy import create_engine, text
from sklearn.preprocessing import OneHotEncoder
from surprise.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from surprise.model_selection import train_test_split as tts2

In [3]:
load_dotenv()
DB_URL = os.getenv("DB_URL")
engine = create_engine(DB_URL)

try:
    connection = engine.connect()
    print("Successfully connected to the database")
except Exception as e:
    print(f"Failed to connect to the database: {e}")

print(f"SQLAlchemy version: {sqlalchemy.__version__}")

Successfully connected to the database
SQLAlchemy version: 2.0.21


In [4]:
query = text('SELECT * FROM epic_5')
df = pd.read_sql_query(query, connection)
df.head()

,PersoonId,CampagneId,aantal_sessies,aantal_bezoeken,SessieThema,SoortCampagne,TypeCampagne,ThemaDuurzaamheid,ThemaFinancieelFiscaal,ThemaInnovatie,ThemaInternationaalOndernemen,ThemaMobiliteit,ThemaOmgeving,ThemaSalesMarketingCommunicatie,ThemaStrategieEnAlgemeenManagement,ThemaTalent,ThemaWelzijn
0,111AF280-8E24-E911-80FB-001DD8B72B62,B5D28043-9F4C-E911-80FC-001DD8B72B62,1,0,Netwerking,Offline,Netwerkevenement,0,0,0,0,0,0,0,0,0,0
1,3A82E7B4-4A57-E811-80F0-001DD8B72B62,7B62B114-920B-EA11-8107-001DD8B72B61,1,0,Netwerking,Offline,Netwerkevenement,0,0,0,0,0,0,0,0,0,0
2,C7FEB15E-FC67-E111-A00F-00505680000A,2D2494E9-0822-E811-80F0-001DD8B72B61,2,0,Netwerking,Offline,Netwerkevenement,1,0,0,0,1,0,0,1,0,0
3,F4FD0816-1468-E111-A00F-00505680000A,DF45C30C-A7F6-E711-80EF-001DD8B72B61,1,0,Haven,Offline,Netwerkevenement,0,0,0,0,0,0,0,0,0,0
4,FAD0814C-38A4-EB11-811A-001DD8B72B61,EEA0ED13-69E4-EB11-8121-001DD8B72B61,1,2,Welt 2.0,Offline,Project,1,0,0,0,0,0,0,0,1,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52567 entries, 0 to 52566
Data columns (total 17 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   PersoonId                           52567 non-null  object
 1   CampagneId                          52567 non-null  object
 2   aantal_sessies                      52567 non-null  int64 
 3   aantal_bezoeken                     52567 non-null  int64 
 4   SessieThema                         52567 non-null  object
 5   SoortCampagne                       52567 non-null  object
 6   TypeCampagne                        52567 non-null  object
 7   ThemaDuurzaamheid                   52567 non-null  int64 
 8   ThemaFinancieelFiscaal              52567 non-null  int64 
 9   ThemaInnovatie                      52567 non-null  int64 
 10  ThemaInternationaalOndernemen       52567 non-null  int64 
 11  ThemaMobiliteit                     52567 non-null  in

In [6]:
# WORK IN PROGRESS UNDER THIS LINE

In [7]:
# CONTENT BASED FILTERING MODEL

In [8]:
# kijken welke items op elkaar lijken op basis van campagne / sessie informatie

In [9]:
df_tfidf = df

df_tfidf['CampagneSessieInformatie'] = df_tfidf['SessieThema'] + ' ' + df_tfidf['SoortCampagne'] + ' ' + df_tfidf['TypeCampagne']

df_tfidf = df_tfidf[['PersoonId', 'CampagneId', 'aantal_sessies', 'CampagneSessieInformatie']]

df_tfidf.head()

,PersoonId,CampagneId,aantal_sessies,CampagneSessieInformatie
0,111AF280-8E24-E911-80FB-001DD8B72B62,B5D28043-9F4C-E911-80FC-001DD8B72B62,1,Netwerking Offline Netwerkevenement
1,3A82E7B4-4A57-E811-80F0-001DD8B72B62,7B62B114-920B-EA11-8107-001DD8B72B61,1,Netwerking Offline Netwerkevenement
2,C7FEB15E-FC67-E111-A00F-00505680000A,2D2494E9-0822-E811-80F0-001DD8B72B61,2,Netwerking Offline Netwerkevenement
3,F4FD0816-1468-E111-A00F-00505680000A,DF45C30C-A7F6-E711-80EF-001DD8B72B61,1,Haven Offline Netwerkevenement
4,FAD0814C-38A4-EB11-811A-001DD8B72B61,EEA0ED13-69E4-EB11-8121-001DD8B72B61,1,Welt 2.0 Offline Project


In [10]:
# VANAF HIER ONZEKER

In [11]:
vectorizer = TfidfVectorizer()

campagne_sessie_matrix = vectorizer.fit_transform(df_tfidf['CampagneSessieInformatie'])

In [12]:
from sklearn.decomposition import TruncatedSVD

# Assuming tfidf_matrix is your TF-IDF matrix
svd = TruncatedSVD(n_components=40)  # Reduce to 100 dimensions
reduced_campagne_sessie_matrix = svd.fit_transform(campagne_sessie_matrix)

In [13]:
content_model = cosine_similarity(reduced_campagne_sessie_matrix)

In [14]:
# WORK IN PROGRESS ABOVE THIS LINE

In [15]:
# COLLABORATIVE FILTERING MODEL

In [16]:
df_svd = df[['PersoonId', 'CampagneId', 'aantal_sessies']]
df_svd.shape

(52567, 3)

In [17]:
min_aantal_sessies = df_svd.loc[df_svd['aantal_sessies'] > 2]
min_aantal_sessies.shape

(6512, 3)

In [18]:
min_aantal_sessies['aantal_sessies'].value_counts()

3     1372
4     1064
6     1048
5      837
7      466
10     307
11     274
14     212
13     195
12     185
8      183
9      149
15     125
17      58
18      14
16      14
26       2
27       2
20       2
22       2
24       1
Name: aantal_sessies, dtype: int64

In [19]:
df_svd = min_aantal_sessies

In [20]:
train_set, test_set = train_test_split(df_svd, test_size=0.25, random_state=7)

In [21]:
reader = Reader(rating_scale=(df_svd['aantal_sessies'].min(), df_svd['aantal_sessies'].max()))

df_svd_surprise = Dataset.load_from_df(df_svd[['PersoonId', 'CampagneId', 'aantal_sessies']], reader)

df_svd_train = Dataset.load_from_df(train_set[['PersoonId', 'CampagneId', 'aantal_sessies']], reader)

In [22]:
param_grid = {"n_epochs": [5, 10, 15], "lr_all": [0.02, 0.025, 0.03], "reg_all": [0.1, 0.2, 0.4], "n_factors": [5, 15, 25]}
grid_search_SVD = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

grid_search_SVD.fit(df_svd_train)

print('Best RMSE:', grid_search_SVD.best_score['rmse'], grid_search_SVD.best_params['rmse'])
print('Best MAE:', grid_search_SVD.best_score['mae'], grid_search_SVD.best_params['mae'])

Best RMSE: 1.5641364172517676 {'n_epochs': 15, 'lr_all': 0.03, 'reg_all': 0.1, 'n_factors': 5}
Best MAE: 1.00948643152618 {'n_epochs': 15, 'lr_all': 0.03, 'reg_all': 0.1, 'n_factors': 5}


In [23]:
trainset, testset = tts2(df_svd_surprise, test_size=0.25, random_state=7)

model_SVD = grid_search_SVD.best_estimator['rmse']

model_SVD.fit(trainset)

predictions = model_SVD.test(testset)

ResultCatcher = pd.DataFrame(predictions, columns=['PersoonId', 'CampagneId', 'Real_Rating', 'Estimated_Rating', 'details'])
ResultCatcher.drop(['details'], axis=1, inplace=True)

In [24]:
ResultCatcher['Estimation_Rounded'] = ResultCatcher.apply(lambda row: round(row.Estimated_Rating), axis=1)
ResultCatcher['Error'] = abs(ResultCatcher['Real_Rating'] - ResultCatcher['Estimation_Rounded'])
ResultCatcher.head()

,PersoonId,CampagneId,Real_Rating,Estimated_Rating,Estimation_Rounded,Error
0,0BB3E091-1C68-E111-A00F-00505680000A,2D21A437-130D-EA11-8107-001DD8B72B62,17.0,13.930845,14,3.0
1,0121FA7B-7C6B-E611-80DE-001DD8B72B61,19C23572-4E56-EB11-8117-001DD8B72B61,5.0,5.832748,6,1.0
2,484C3848-0B59-E811-80F0-001DD8B72B62,13C5880B-3AE9-EB11-8121-001DD8B72B61,3.0,4.129787,4,1.0
3,0CB09A92-F4F2-ED11-8849-6045BD895499,044F17B7-A8B6-ED11-83FF-6045BD895CDC,4.0,4.276749,4,0.0
4,5750E318-E967-E111-A00F-00505680000A,B6F561C5-5D33-E811-80F0-001DD8B72B61,6.0,6.157800,6,0.0


In [25]:
sum_of_real_rating = ResultCatcher['Real_Rating'].sum()
sum_of_error = ResultCatcher['Error'].sum()
count_of_real_rating = ResultCatcher['Real_Rating'].count()
count_of_correct_predictions = ResultCatcher['Real_Rating'][ResultCatcher['Error'] == 0].count()

error_percentage = (sum_of_error / sum_of_real_rating) * 100
accuracy_percentage = (count_of_correct_predictions / count_of_real_rating) * 100

print("Sum of Real_Rating: ", sum_of_real_rating)
print("Sum of Error: ", sum_of_error)
print("Count of Real_Rating: ", count_of_real_rating)
print("Count of Real_Rating where Error is 0: ", count_of_correct_predictions)

print("Error Percentage: {:.2f}%".format(error_percentage))
print("Accuracy Percentage: {:.2f}%".format(accuracy_percentage))

Sum of Real_Rating:  10513.0
Sum of Error:  1249.0
Count of Real_Rating:  1628
Count of Real_Rating where Error is 0:  806
Error Percentage: 11.88%
Accuracy Percentage: 49.51%


In [26]:
# https://github.com/EmirKorkutUnal/Python-Surprise-Predictions-on-Custom-Dataset

In [27]:
# WORK IN PROGRESS UNDER THIS LINE

In [28]:
# beide modellen combineren, content based eerst dan collaborative filtering
# content based voor 'user likes x so user will like y' omdat het op elkaar lijkt op basis van campagne / sessie informatie
# lost het 'cold start' probleem op denk ik

In [29]:
# functie maken die voor een campagne de top 20 personen geeft die het meest waarschijnlijk zijn om de campagne te volgen
# enkel als de persoon de campagne nog niet heeft gevolgd

In [30]:
# WORK IN PROGRESS ABOVE THIS LINE